## Baixar dados de 1 periodo especifico

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Função para obter o token de acesso
def get_token():
    url = "https://www.ana.gov.br/hidrowebservice/EstacoesTelemetricas/OAUth/v1"
    
    headers = {
        "accept": "*/*",
        "Identificador": "LOGIN",  # Substitua com o seu identificador
        "Senha": "SENHA"  # Substitua com a sua senha
        # O loguin e senha foram subistiruido para segurança
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Retorna o token de autenticação
        data = response.json()
        token = data["items"]["tokenautenticacao"]
        return token
    else:
        raise Exception(f"Erro ao obter token: {response.status_code}, {response.text}")

# Função para fazer o download dos dados detalhados de uma estação
def baixar_dados_detalhados(codigo_estacao, data_inicio, data_fim, token):
    url = f"https://www.ana.gov.br/hidrowebservice/EstacoesTelemetricas/HidroinfoanaSerieTelemetricaDetalhada/v1"
    
    params = {
        'Código da Estação': codigo_estacao,
        'Tipo Filtro Data': 'DATA_LEITURA',
        'Data de Busca (yyyy-MM-dd)': data_inicio.strftime('%Y-%m-%d'),
        'Range Intervalo de busca': 'HORA_24'  # Ajuste para resolução horária
    }
    
    headers = {
        'accept': '*/*',
        'Authorization': f'Bearer {token}'
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao baixar dados da estação {codigo_estacao}: {response.status_code}")
        return None

# Função para baixar e filtrar os dados detalhados por estação
def baixar_e_filtrar_estacao(codigo_estacao, inicio, fim):
    token = get_token()  # Obter o token de autenticação
    df_estacao = pd.DataFrame()  # Dataframe vazio para a estação específica
    
    data_atual = inicio
    while data_atual <= fim:
        data_fim = min(data_atual + timedelta(days=1), fim)
        dados = baixar_dados_detalhados(codigo_estacao, data_atual, data_fim, token)
        
        if dados and 'items' in dados:
            # Converter os dados em um dataframe
            df_temp = pd.DataFrame(dados['items'])
            df_estacao = pd.concat([df_estacao, df_temp], ignore_index=True)  # Adiciona os dados ao dataframe da estação
            
        data_atual = data_fim + timedelta(seconds=1)
    
    return df_estacao

# Função principal para baixar os dados de várias estações
def baixar_dados_varias_estacoes(estacoes, inicio, fim):
    for estacao in estacoes:
        df_estacao = baixar_e_filtrar_estacao(estacao, inicio, fim)
        
        # Salvar cada estação em um arquivo CSV separado
        df_estacao.to_csv(f'dados_estacao_detalhada_{estacao}-{ano}.csv', index=False)
        print(f"Dados detalhados da estação {estacao} salvos no arquivo dados_estacao_detalhada_{estacao}-{ano}.csv")

# Definir as estações e o período para download
#estacoes = ['86306000', '86510000', '86743800','86195000', '2852004', '86520000','86390900', '86490500', '86406000','2851044', '86117000', '86060010','86100600', '86213000']  # Lista de códigos de estações
estacoes = ['86510000 ']  # Lista de códigos de estações
ano = 2024
data_inicio = datetime(ano, 1, 1)
data_fim = datetime(ano, 9, 1) # Período para teste

# Baixar os dados de várias estações
baixar_dados_varias_estacoes(estacoes, data_inicio, data_fim)


Dados detalhados da estação 86510000  salvos no arquivo dados_estacao_detalhada_86510000 -2024.csv


## Baixa dados de vários anos

In [10]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Função para obter o token de acesso
def get_token():
    url = "https://www.ana.gov.br/hidrowebservice/EstacoesTelemetricas/OAUth/v1"
    
    headers = {
        "accept": "*/*",
        "Identificador": "LOGIN",  # Substitua com o seu identificador
        "Senha": "SENHA"  # Substitua com a sua senha
        # O loguin e senha foram subistiruido para segurança
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Retorna o token de autenticação
        data = response.json()
        token = data["items"]["tokenautenticacao"]
        return token
    else:
        raise Exception(f"Erro ao obter token: {response.status_code}, {response.text}")

# Função para fazer o download dos dados detalhados de uma estação
def baixar_dados_detalhados(codigo_estacao, data_inicio, data_fim, token):
    url = f"https://www.ana.gov.br/hidrowebservice/EstacoesTelemetricas/HidroinfoanaSerieTelemetricaDetalhada/v1"
    
    params = {
        'Código da Estação': codigo_estacao,
        'Tipo Filtro Data': 'DATA_LEITURA',
        'Data de Busca (yyyy-MM-dd)': data_inicio.strftime('%Y-%m-%d'),
        'Range Intervalo de busca': 'HORA_24'  # Ajuste para resolução horária
    }
    
    headers = {
        'accept': '*/*',
        'Authorization': f'Bearer {token}'
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao baixar dados da estação {codigo_estacao}: {response.status_code}")
        return None

# Função para baixar e filtrar os dados detalhados por estação
def baixar_e_filtrar_estacao(codigo_estacao, inicio, fim):
    token = get_token()  # Obter o token de autenticação
    df_estacao = pd.DataFrame()  # Dataframe vazio para a estação específica
    
    data_atual = inicio
    while data_atual <= fim:
        data_fim = min(data_atual + timedelta(days=1), fim)
        dados = baixar_dados_detalhados(codigo_estacao, data_atual, data_fim, token)
        
        if dados and 'items' in dados:
            # Converter os dados em um dataframe
            df_temp = pd.DataFrame(dados['items'])
            df_estacao = pd.concat([df_estacao, df_temp], ignore_index=True)  # Adiciona os dados ao dataframe da estação
            
        data_atual = data_fim + timedelta(seconds=1)
    
    return df_estacao

# Função para baixar os dados de um determinado ano
def baixar_dados_por_ano(estacoes, ano):
    for estacao in estacoes:
        data_inicio = datetime(ano, 1, 1)  # Início do ano
        data_fim = datetime(ano, 12, 31)   # Fim do ano
        df_estacao = baixar_e_filtrar_estacao(estacao, data_inicio, data_fim)
        
        # Salvar cada estação em um arquivo CSV separado com o nome do ano
        df_estacao.to_csv(f'dados_estacao_detalhada_{estacao}-{ano}.csv', index=False)
        print(f"Dados detalhados da estação {estacao} para o ano {ano} salvos no arquivo dados_estacao_detalhada_{estacao}-{ano}.csv")

# Definir as estações e os anos para download
estacoes = ['86520100']  # Lista de códigos de estações

# Executar o download para cada ano
anos = [2018, 2019, 2020, 2021, 2022, 2023]
for ano in anos:
    baixar_dados_por_ano(estacoes, ano)


Dados detalhados da estação 86520100 para o ano 2020 salvos no arquivo dados_estacao_detalhada_86520100-2020.csv
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixar dados da estação 86520100: 401
Erro ao baixa